# 這個程式專產白卷......

## Import packages

In [24]:
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

## Read data

In [25]:
test = pd.read_csv('data/cleaned_data/test_sol24.csv')
test.review = test.review.astype(str)

In [26]:
test['review'] = test['review'].str.strip().str.lower().str.replace(r'\u200b', ' ')

## All-five

In [ ]:
all_five = [5]*len(test)
submission = test.drop('review', axis=1)
submission['rating'] = all_five
submission

In [ ]:
submission.to_csv('submission/submission_all-five.csv', index=False)

## Negative Detection

In [27]:
neg_words = []
with open('data/lib/neg_words.txt', encoding='utf8') as f:
    for word in f.readlines():
        neg_words.append(word.strip())

f.close()
neg_words

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [28]:
len(neg_words)

4783

In [29]:
# 差不多會跑2分半，看得到進度比較心安
from tqdm import tqdm

test['rating'] = [5]*len(test)
neg_num_arr = np.array([0]*len(test))
for nw in tqdm(neg_words):
    neg_index = test[test['review'].str.contains(nw.replace('*', ''))].index
    neg_num_arr[neg_index] += 1

test['neg_num'] = neg_num_arr

100%|██████████| 4783/4783 [02:33<00:00, 31.14it/s]


In [30]:
bad_reivews = test[test['neg_num'] > 1]

In [31]:
import emojis
positive_keys = ['good', 'excellent', 'bgus', 'bagus', 'love', 'perfect', 'great', 'fast', 'reasonable', '❤', '😁', '😄', '😊', '🥰', '😍', '😘', '👍', '👍', '🤩', '💋', '😻', '❤️', '💙', '💓', '💚']

for k in positive_keys:
    k = emojis.decode(k)
    bad_reivews.drop(bad_reivews[bad_reivews['review'].str.contains(k)].index, inplace=True)

len(bad_reivews)

C:\Users\pb580\anaconda3\envs\ML_Env\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


7263

In [32]:
bad_reivews

,review_id,review,rating,neg_num
14,15,"to run through the shop to buy always the rain too, put his little guilty shopee delivery.\npackaging carefully, not been smashed, not spilled out",5,6
17,18,uda waiting lma unfortnately his reply on messages depending sma,5,4
30,31,the quality of the product is according to the expectations and fits very well works as well as the ones that i am using right know,5,3
48,49,"unfortunately, this beautiful out of stock seller is very careful handling intentions recommended!",5,2
54,55,this is by design of course but it may prevent the toe box area from limbering up enough to allow comfortable splay in the ball of one foot,5,2
...,...,...,...,...
60415,60416,"i think the product is so nice, but i will say that the gems falls off quickly, it is understandable because of the price. other than that i’m satisfied so i will still give 5 stars ^_^",5,2
60417,60418,they did not even really need to be broken in .,5,2
60418,60419,"quality dirty towels were just okay, mentng2 he bought at a bargain price, do not know can not lose to dirty tu ..",5,4
60420,60421,buy tissue for suppies at home. thank ya delivery cepet,5,2


In [33]:
test.loc[bad_reivews.index, 'rating'] = 1

In [34]:
submission = test.drop(['review', 'neg_num'], axis=1)
submission.to_csv('submission/submission_15.csv', index=False)